In [34]:
import os
import json
import math

In [47]:
# File path
working_dir = os.path.dirname(os.path.abspath(''))
data_process_dir = working_dir + "\\src\\data\\processing-batch\\data-process"
stop_word_file = working_dir + "\\src\\data\\processing-batch\\vietnamese-stopwords-dash.txt"
v_json_export_path = working_dir + "\\src\\data\\processing-batch\\topic_model.json"
pmi_json_export_path = working_dir + "\\src\\data\\processing-batch\\pmi.json"

In [16]:
# Load topic model 
with open(v_json_export_path, "r", encoding="utf-8") as file:
    topic_model = json.load(file)

In [3]:
extension = ".txt.pro"
delimiter_text = "/"
noun_type = "N"
topics = [
    "ThoiSu",
    "TheGioi",
    "KinhDoanh",
    "BatDongSan",
    "KhoaHoc",
    "GiaiTri",
    "TheThao",
    "PhapLuat",
    "GiaoDuc",
    "SucKhoe",
    "DoiSong",
    "DuLich",
    "SoHoa",
    "Xe",
]

In [4]:
# Lấy ra các hư từ
stop_word_file = open(stop_word_file, "r", encoding="utf-8")
stop_words = stop_word_file.read()
stop_words = stop_words.split("\n")
stop_words.append("ảnh")

In [19]:
# Lấy ra các core word từ topic model
core_word = []
for key, items in topic_model.items():
    for item in items:
        core_word.append(item[0])

In [23]:
def get_noun_word(words: list):
    noun_words = []
    for text_split in words:
        text_split = text_split
        text_and_type = text_split.split(delimiter_text)
        type = text_and_type.pop()
        if type == noun_type:
            text = text_and_type.pop()
            # Nếu từ không phải hư từ thì đưa vào tập V
            if text.lower() not in stop_words and text.lower() in core_word:
                noun_words.append(text.lower())
    return noun_words

In [13]:
S = []

In [14]:
# Xử lý lấy tất cả các câu đưa vào tập S
for topic in topics:
    data_process_file_path = data_process_dir + f"\\{topic}{extension}"
    with open(data_process_file_path, "r", encoding="utf-8") as file:
        text = file.read()
        # Xoá các ký tự xuống dòng
        text = text.replace("\n", " ")
        
        # Tách câu
        sentences_splits = text.split("./.")
        
        # Đưa các câu vào tập S
        S = S + sentences_splits

In [55]:
# Tính toán số lần xuất hiện của các cặp từ 
pair_noun = {}
count_noun = {}
count_all_word = 0
for sentence in S:
    words_in_sen = sentence.split(" ")
    noun_word = get_noun_word(words_in_sen)
    noun_word.sort()
    
    # Tính số lượng các từ chủ đề trong tập huấn luyện
    count_all_word += len(noun_word)
    for i in range(len(noun_word)):
        # Tính số lần xuất hiện của từ
        if noun_word[i] not in count_noun.keys():
            count_noun[f'{noun_word[i]}'] = 1
        else:
            count_noun[f'{noun_word[i]}'] += 1
        
        for j in range(i, len(noun_word)):
            if noun_word[i] != noun_word[j]:
                # Tính số lần xuất hiện của các cặp từ
                word_pair = noun_word[i] + " " + noun_word[j]
                if word_pair not in pair_noun.keys():
                    pair_noun[f'{word_pair}'] = 1
                else:
                    pair_noun[f'{word_pair}'] += 1

count_noun['count_all_word'] = count_all_word

In [56]:
# Tính toán PMI cho các từ
PMI = {}
count_all_word = count_noun['count_all_word']
for key, item in pair_noun.items():
    word = key.split(" ")
    p_pair = pair_noun[f'{key}'] / count_all_word
    p_word1 = count_noun[f'{word[0]}'] / count_all_word
    p_word2 = count_noun[f'{word[1]}'] / count_all_word
    
    pmi = math.log2(p_pair / (p_word1 * p_word2))
    if pmi > 0:
        PMI[f'{key}'] = pmi

In [57]:
PMI

{'chỗ miền': 1.9370072925019386,
 'chỗ trung_lương': 6.639967002621546,
 'chỗ tốc': 5.398958903117751,
 'chỗ ôtô': 6.9974820836990075,
 'miền trung_lương': 8.2589353873893,
 'miền tốc': 7.017927287885506,
 'miền ôtô': 2.1439626970040186,
 'trung_lương tốc': 11.720886998005113,
 'trung_lương ôtô': 6.524994312236264,
 'tốc ôtô': 6.2839862127324695,
 'chiều huyện': 4.724138250881769,
 'chiều xe_tải': 5.6440714009542825,
 'chiều xã': 4.341763501536208,
 'chiều ôtô': 4.409517094816328,
 'chiều đuôi': 3.591603981060147,
 'chiều địa_bàn': 5.03870268957129,
 'huyện xe_tải': 5.246479036134848,
 'huyện xã': 7.6153479324910105,
 'huyện ôtô': 4.284943224403309,
 'huyện đuôi': 3.837867806015437,
 'huyện địa_bàn': 6.391881718443091,
 'xe_tải xã': 5.244246995621945,
 'xe_tải ôtô': 6.765591328950302,
 'xe_tải đuôi': 8.456019434312365,
 'xe_tải địa_bàn': 3.7432468060451183,
 'xã ôtô': 4.424730188762835,
 'xã đuôi': 3.513707670615173,
 'xã địa_bàn': 6.260366660985223,
 'ôtô đuôi': 6.083961604424475,
 'ô

In [58]:
# Export thành file json
# Serializing json
json_object = json.dumps(PMI, indent=4)
 
# Writing to sample.json
with open(pmi_json_export_path, "w", encoding="utf-8") as outfile:
    json.dump(PMI, outfile)

In [7]:
class Sentence():
    '''Class câu'''
    content: str
    position: int
    topic: str
    weight: float
    document_position: int
    
    def __init__(self, content:str, position:int, topic:str):
      self.content = content
      self.position = position
      self.topic = topic
      self.weight = 0
    
    def set_topic(self, topic:str):
        self.topic = topic
    
    def set_weight(self, weight: float):
        self.weight = weight
    
    def set_document_position(self, document_position: int):
        self.document_position = document_position
    
class Document():
    '''Class văn bản'''
    sentences: list[Sentence]
    topic: str
    weight: float
    position: int
    
    def __init__(self, sentences: list, topic: str, position: int):
      self.sentences = sentences
      self.topic = topic
      self.weight = 0
      self.position = position
    
    def add_sentence(self, sentence: Sentence):
        self.sentences.append(sentence)
    
    def get_document(self):
        document = ''
        for sentence in self.sentences:
            document+=sentence.content
        
        return document
    
    def set_topic(self, topic: str):
        self.topic = topic
    
    def set_weight(self, weight: float):
        self.weight = weight
        
    def calculate_weight(self):
        weight = 0
        for sentence in self.sentences:
            weight += sentence.weight
        self.weight = weight / (len(self.sentences))
    
    def apply_weight_for_sentences(self, weight_sentences: list[dict]):
        for i in range(len(self.sentences)):
            self.sentences[i].set_weight(weight_sentences[i]['weight'])
        
        self.calculate_weight()
    
    def apply_document_weight_for_sentences(self):
        for i in range(len(self.sentences)):
            self.sentences[i].set_weight(self.sentences[i].weight * self.weight)

In [12]:
sen1 = Sentence('',1,'')
sen2 = Sentence('',2,'')

sen1.set_weight(9)
sen2.set_weight(1)


all_sen = [sen1, sen2]
doc1 = Document(all_sen,'',0)

doc1.set_weight(0.1)
doc1.apply_document_weight_for_sentences()

all_sen = sorted(all_sen, key = lambda item: item.weight)
all_sen[0].weight

0.1

In [15]:
import pandas as pd


def split_document(document:pd.DataFrame, cluster_size: int):
    # Shuffle ngẫu nhiên các chỉ số của DataFrame
    shuffled_df = document.sample(frac=1).reset_index(drop=True)
    
    # Chia thành các nhóm gồm cluster_size dòng
    groups = [shuffled_df.iloc[i:i + cluster_size] for i in range(0, len(document), cluster_size)]
    
    # Loại bỏ nhóm cuối nếu không đủ cluster_size dòng
    groups = [group for group in groups if len(group) == cluster_size]
    
    return groups

In [16]:
data = {'A': range(1, 21)}  # DataFrame với 20 dòng
df = pd.DataFrame(data)

# Chia DataFrame thành các cụm ngẫu nhiên, mỗi cụm 5 dòng
groups = split_document(df, 5)

# Hiển thị các cụm
for i, group in enumerate(groups):
    print(f"Group {i+1}:\n", group, "\n")

Group 1:
     A
0  19
1  13
2   8
3  11
4   9 

Group 2:
     A
5  16
6   4
7  14
8  12
9  10 

Group 3:
      A
10   2
11   3
12  17
13   5
14   1 

Group 4:
      A
15  20
16   7
17  15
18   6
19  18 



In [19]:
from itertools import chain


sentences = ["Nom nom nom", "NOM NOM NOM"]
words = [sentence.split(" ") for sentence in sentences]
# Flatten list từ
words = list(chain.from_iterable(words))
words

['Nom', 'nom', 'nom', 'NOM', 'NOM', 'NOM']